# PyJive workshop: FE$^2$

In [ ]:
import sys
import numpy as np
sys.path.append('../../../pyjive')

import main
from utils import proputils as pu
from names import GlobNames as gn

%matplotlib widget

In [ ]:
import contextlib
import os
from urllib.request import urlretrieve

def findfile(fname):
    url = "https://gitlab.tudelft.nl/cm/public/drive/-/raw/main/fe2workshop/" + fname + "?inline=false"
    if not os.path.isfile(fname):
        print(f"Downloading {fname}...")
        urlretrieve(url, fname)

findfile("macro.pro")
findfile("micro.pro")
findfile("dogbone.msh")
findfile("unitCell.msh")

### Run multiscale analysis

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 1: Run predefined problem</b>   

Run the two-scale problem by executing the cell below. The case is that of a dog-bone specimen in uniaxial tension on the macroscale, linked to a unit-cell micromodel with FE$^2$. Coarse meshes are provided for both scales, and only a small number of time steps is executed to limit the runtime.

</p>
</div>

In [ ]:
globdat = main.jive(pu.parse_file('macro.pro'))

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 2: Inspect the code</b>   

Answer the following questions
- Which part of the main input file `macro.pro` defines that a two-scale problem is solved?
- The macroscale input file is read on the notebook, how is the microscale input file processed?
- Can you find how the macrostrain is imposed on the micromodel and how the homogenized microstress is returned to the macroscale? 

</p>
</div>

### Visualize results
The resulting displacement field and its evolution can be visualized on the macroscale

In [ ]:
viewmodule = globdat[gn.MODULEFACTORY].get_module('View','view')

viewprops = { 'view': {}}
viewprops['view']['plot'] = 'solution[dx]'
viewprops['view']['interactive'] = 'True'
viewprops['view']['constantLevels'] = 'False'
viewprops['view']['constantTicks'] = 'True'
viewprops['view']['step0'] = 3
viewprops['view']['ncolors'] = 10

viewmodule.init(viewprops, globdat)
viewmodule.plot(globdat)

And also on the microscale

In [ ]:
microglobdats = globdat[gn.MODEL]._models[0]._mat._globdats
viewmodule.init(viewprops, microglobdats[0])
viewmodule.plot(microglobdats[0])

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 3: Inspect the results</b>   

The cells above generate visualizations from both scales. 

- The microscale visualization is plotting results for one selected micromodel. Can you also plot results for a different one? 
- For the macroscale, the number of time steps agrees with what is specified in the input file. For the microscale, that is not the case. What defines the number of time steps that is plotted for the microscale? (**tip**: Results are stored in `globdat` every time the `NonlinModule` reaches the end of its run function, this is somewhat hidden, but done by the `ControlModule` which is the base class of the `NonlinModule`)

</p>
</div>

<div style="background-color:#AABAB2; color: black; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">
<p>
<b>Task 4: A missing feature in the present implementation</b>   

Although the simulation is defined as a nonlinear one, it remains in the linear regime. In fact, the provided code is not entirely suitable for performing nonlinear simulations. There is an issue with how material history is stored. In the regular nonlinear finite element workflow in `pyjive`, the action `COMMIT` takes care of storing history. 

- At what point in the nonlinear solution algorithm is the `COMMIT` action taken for a single scale simulation? 
- What would need to be done to get consistent behavior for  a nonlinear two-scale simulation?

</p>
</div>